In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [4]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [6]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=20)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['fractal'].value_counts())

0.0    55433
1.0     9351
2.0     8978
Name: fractal, dtype: int64


In [8]:
value_per_pip = 1.0
max_pips_to_risk = 0.0050
amounts_per_day = [-4, -5, -6]

In [16]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def pip_round(x, base=0.0005):
  x_round = round(base * round(float(x)/base),4)
  if x_round < x:
      return x_round + base
  else:
      return x_round

def get_end_date(j):
  return df.loc[df.index[j + 1], 'Date'] if j + 1 < len(df) else None

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, use_tl):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    i = 18

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        atr = df.loc[df.index[i - 1], 'atr']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        prev_mo = df.loc[df.index[i - 1], 'Mid_Open']
        prev_mc = df.loc[df.index[i - 1], 'Mid_Close']
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [0, macd2, macdsignal2, macd1, macdsignal1]
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        if trade is None:
          if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml2 > ema200 and curr_ml1 > ema200 and max([0, macd1, macd2, macdsignal1, macdsignal2]) == 0 and enough_volatility and trending and curr_bar_length <= bar_length:
          # if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml2 > ema200 and curr_ml1 > ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_buys = True

              if all_buys:
                open_price = float(curr_ask_open)
                pullback = None
                # u = i - 3

                # while u >= 0:
                #     if df.loc[df.index[u], 'fractal'] == 1:
                #         curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                #         curr_fractal_distance = curr_ao - curr_pullback

                #         if curr_fractal_distance >= fractal_distance:
                #             pullback = curr_pullback
                #             break

                #     u -= 1
                prev_highs = df.loc[df.index[i - 18:i], 'Mid_High']
                prev_lows = df.loc[df.index[i - 18:i], 'Mid_Low']
                pips_diff = pip_round(abs(max(prev_highs) - min(prev_lows)))
                pullback = open_price - pips_diff

                if pullback is not None:
                    stop_loss = round(pullback - pullback_cushion, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh2 < ema200 and curr_mh1 < ema200 and min([0, macd1, macd2, macdsignal1, macdsignal2]) == 0 and enough_volatility and trending and curr_bar_length <= bar_length:
          # elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh2 < ema200 and curr_mh1 < ema200 and enough_volatility and trending and curr_bar_length <= bar_length:    
              all_sells = True

              if all_sells:
                open_price = float(curr_bid_open)
                pullback = None
                # u = i - 3

                # while u >= 0:
                #     if df.loc[df.index[u], 'fractal'] == 2:
                #         curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                #         curr_fractal_distance = curr_pullback - curr_bo

                #         if curr_fractal_distance >= fractal_distance:
                #             pullback = curr_pullback
                #             break

                #     u -= 1
                prev_highs = df.loc[df.index[i - 18:i], 'Mid_High']
                prev_lows = df.loc[df.index[i - 18:i], 'Mid_Low']
                pips_diff = pip_round(abs(max(prev_highs) - min(prev_lows)))
                pullback = open_price + pips_diff

                if pullback is not None:
                    stop_loss = round(pullback + pullback_cushion, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
              trade['stop_loss'] = curr_bid_high - trade['pips_risked']

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if use_tl:
          if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
            trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [17]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1, 1.5, 2]
# risk_reward_ratio_vals = [2]
# pullback_cushion_vals = [0, 5]
pullback_cushion_vals = [0]
# fractal_distances = [5, 10, 15, 20]
fractal_distances = [0]
bar_lengths = [5, 10, 50]
# bar_lengths = [5]
adx_cutoffs = [15, 20, 25]
# adx_cutoffs = [20]
spread_cutoffs = [2, 3]
# spread_cutoffs = [2]
tl_vals = [True, False]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(fractal_distances) * len(bar_lengths) * len(adx_cutoffs) * len(spread_cutoffs) * len(tl_vals)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for fractal_distance in fractal_distances:
      for bar_length in bar_lengths:
        for adx_cutoff in adx_cutoffs:
          for spread_cutoff in spread_cutoffs:
            for tl_val in tl_vals:
              all_combos.append((risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, tl_val))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_adx_cutoff = None
best_spread_cutoff = None
best_tl_val = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, tl_val in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, tl_val)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'pullback': pullback_cushion, 'fractal_dist': fractal_distance, 'bar_length': bar_length, 'adx': adx_cutoff, 'spread': spread_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length
    best_adx_cutoff = adx_cutoff
    best_spread_cutoff = spread_cutoff
    best_tl_val = tl_val

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 108

-181.50000000001864
Num buys: 120
Num sells: 116
Num trades: 236
Num wins: 89
Num losses: 147
Win streak: 5
Loss streak: 9
Remaining runs: 107
Best reward so far: -181.50000000001864

-3323.999999999981
Num buys: 156
Num sells: 151
Num trades: 307
Num wins: 91
Num losses: 216
Win streak: 5
Loss streak: 15
Remaining runs: 106
Best reward so far: -181.50000000001864

935.0000000000164
Num buys: 40
Num sells: 46
Num trades: 86
Num wins: 44
Num losses: 42
Win streak: 7
Loss streak: 7
Remaining runs: 105
Best reward so far: 935.0000000000164

-2407.9999999999795
Num buys: 164
Num sells: 162
Num trades: 326
Num wins: 119
Num losses: 207
Win streak: 6
Loss streak: 10
Remaining runs: 104
Best reward so far: 935.0000000000164

-2296.999999999997
Num buys: 111
Num sells: 115
Num trades: 226
Num wins: 104
Num losses: 122
Win streak: 5
Loss streak: 6
Remaining runs: 103
Best reward so far: 935.0000000000164

-392.49999999997794
Num buys: 41
Num sells: 46
Num trades: 87
Num wins: 31


In [18]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))
print('Best spread: ' + str(best_spread_cutoff))
print('Best adx: ' + str(best_adx_cutoff))
print('Best TL val: ' + str(best_tl_val))
print('-----------------------')
print('Top results:')
for entry in best_rewards:
    print(entry)

# 2018-2019:
# {'reward': 1539, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 1554, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 575, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': -94, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 553, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 526, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 153, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1345, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 585, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': -99, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 2}
# {'reward': 150, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': -94, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 1355, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': -96, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 3}
# {'reward': 115, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 102, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': -100, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -125, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': -92, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': -139, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}

# 2019-2020:
# {'reward': 494, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 1306, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1316, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1088, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1345, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 892, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 801, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 562, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 495, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 895, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -34, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1077, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1323, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -17, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 741, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 549, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 742, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 797, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 350, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 348, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}

# {'reward': 1788, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1795, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 803, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1211, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1335, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 889, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 1329, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 889, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 728, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 787, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1042, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 1431, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 728, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1320, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1425, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1057, 'ratio': 2, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 1089, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1341, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1222, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1093, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}


# 2020-2021:
# {'reward': 2381, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 15, 'spread': 3}
# {'reward': 2239, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2513, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2666, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 10}
# {'reward': 2254, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 10}
# {'reward': 2516, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 10}
# {'reward': 2256, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2277, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 2254, 'ratio': 2, 'pullback': 0, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 2366, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 15, 'spread': 2}
# {'reward': 2531, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2552, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 2285, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2667, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 2238, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 10}
# {'reward': 2280, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 10}
# {'reward': 2552, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 10}
# {'reward': 2369, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 15, 'spread': 10}
# {'reward': 2673, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 2523, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}

# {'reward': 549, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1254, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 717, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 734, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 2}
# {'reward': 560, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 517, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 528, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 503, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 3}
# {'reward': 729, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 732, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 25, 'spread': 3}
# {'reward': 642, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 710, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 495, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 655, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 779, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1259, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 50, 'adx': 20, 'spread': 2}
# {'reward': 725, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 772, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 674, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 664, 'ratio': 1.5, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}


------------ FINAL RESULTS ------------
Best reward: 935.0000000000164
Best risk/reward ratio: 1
Best pullback cushion: 0
Best fractal distance: 0
Best bar length: 5
Best spread: 2
Best adx: 25
Best TL val: False
-----------------------
Top results:
{'reward': -181, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 15, 'spread': 3}
{'reward': 935, 'ratio': 1, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 25, 'spread': 2}
{'reward': -392, 'ratio': 1, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 25, 'spread': 2}
{'reward': -21, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 20, 'spread': 3}
{'reward': -26, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 20, 'spread': 2}
{'reward': -418, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 25, 'spread': 3}
{'reward': -165, 'ratio': 1.5, 'pullback': 0, 'fractal_dist': 0, 'bar_length': 5, 'adx': 15, 'spread': 2}
{'reward': -417

In [11]:
print(df['macd'].min())
print(df['macd'].max())
print(df['macd'].mean())

-0.003969309682979816
0.0035649639441031056
8.15201580071511e-06


In [17]:
print(df['macdsignal'].min())
print(df['macdsignal'].max())
print(df['macdsignal'].mean())

-0.0036593454070104913
0.003094390540288942
8.13115151206653e-06
